In [11]:
import numpy as np
import tensorflow as tf

from recommender import Recommender
import tensorflow_recommenders as tfrs

import pandas as pd


In [2]:
test_ds = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/test").cache()

portfolios = tf.data.Dataset.load("D:/dev work/recommender systems/Atrad_CARS/data/portfolios_tfds").cache()

items_ids = portfolios.batch(10000).map(lambda x: x["STOCKCODE"])
item_names = portfolios.batch(10000).map(lambda x: x["STOCKNAME"])
item_GICS = portfolios.batch(10000).map(lambda x: x["GICS"])

user_ids = portfolios.batch(10000).map(lambda x: x["CDSACCNO"])

unique_item_ids = np.unique(np.concatenate(list(items_ids)))
unique_item_names = np.unique(np.concatenate(list(item_names)))
unique_item_gics = np.unique(np.concatenate(list(item_GICS)))

unique_user_ids = np.unique(np.concatenate(list(user_ids)))

# need these to initialize timestamp embedding layers in future steps

timestamps = np.concatenate(list(portfolios.map(lambda x: x["UNIX_TS"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)




Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [4]:
loaded_model = Recommender(
    use_timestamp = True,
    portfolios = portfolios
)


In [6]:
loaded_model.load_weights("D:/dev work/recommender systems/ATRAD_CARS/model_weights/2024_04_26/tf_retrival_2024_04_26_15_17")

loaded_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

loaded_model.evaluate(test_ds.shuffle(100_000).batch(1024), return_dict=True)

1/1 [==============================] - 0s 268ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0156 - factorized_top_k/top_5_categorical_accuracy: 0.0169 - factorized_top_k/top_10_categorical_accuracy: 0.0169 - factorized_top_k/top_50_categorical_accuracy: 0.0377 - factorized_top_k/top_100_categorical_accuracy: 0.0676 - loss: 5383.4829 - regularization_loss: 0.0000e+00 - total_loss: 5383.4829


{'factorized_top_k/top_1_categorical_accuracy': 0.015604681335389614,
 'factorized_top_k/top_5_categorical_accuracy': 0.01690507121384144,
 'factorized_top_k/top_10_categorical_accuracy': 0.01690507121384144,
 'factorized_top_k/top_50_categorical_accuracy': 0.03771131485700607,
 'factorized_top_k/top_100_categorical_accuracy': 0.06762028485536575,
 'loss': 5383.48291015625,
 'regularization_loss': 0,
 'total_loss': 5383.48291015625}

In [9]:
items = portfolios.map(lambda x: {
'STOCKCODE' : x['STOCKCODE'],
'STOCKNAME' : x['STOCKNAME'],
'GICS' : x['GICS'],

})

In [12]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(loaded_model.user_model)

In [14]:
items.batch(100).map(loaded_model.item_model)

AttributeError: Exception encountered when calling layer 'item_model' (type ItemModel).

in user code:

    File "d:\dev work\recommender systems\Atrad_CARS\code\retrieval\item_embedding.py", line 74, in call  *
        axis = 1)
    File "c:\Users\bpadmin\anaconda3\envs\atrad_cars_v2\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\bpadmin\anaconda3\envs\atrad_cars_v2\lib\site-packages\keras\engine\functional.py", line 644, in _run_internal_graph
        input_t._keras_mask = mask

    AttributeError: Exception encountered when calling layer 'sequential' (type Sequential).
    
    'str' object has no attribute '_keras_mask'
    
    Call arguments received by layer 'sequential' (type Sequential):
      • inputs='STOCKCODE'
      • training=None
      • mask=None


Call arguments received by layer 'item_model' (type ItemModel):
  • inputs={'STOCKCODE': 'tf.Tensor(shape=(None,), dty

In [13]:
# recommends item out of the entire items dataset.
index.index(items.batch(100).map(loaded_model.item_model), items)

AttributeError: Exception encountered when calling layer 'item_model' (type ItemModel).

in user code:

    File "d:\dev work\recommender systems\Atrad_CARS\code\retrieval\item_embedding.py", line 74, in call  *
        axis = 1)
    File "c:\Users\bpadmin\anaconda3\envs\atrad_cars_v2\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\bpadmin\anaconda3\envs\atrad_cars_v2\lib\site-packages\keras\engine\functional.py", line 644, in _run_internal_graph
        input_t._keras_mask = mask

    AttributeError: Exception encountered when calling layer 'sequential' (type Sequential).
    
    'str' object has no attribute '_keras_mask'
    
    Call arguments received by layer 'sequential' (type Sequential):
      • inputs='STOCKCODE'
      • training=None
      • mask=None


Call arguments received by layer 'item_model' (type ItemModel):
  • inputs={'STOCKCODE': 'tf.Tensor(shape=(None,), dty

In [10]:

# recommends item out of the entire items dataset.
index.index(items.batch(100).map(loaded_model.item_model), items)

# Get recommendations.
j = str(40)
_, titles = index(tf.constant([j]))
print(f"Recommendations for user %s: {titles[0]}" %(j))

NameError: name 'tfrs' is not defined